In [1]:
import one_minus_utils.azure_utils as azutils
import one_minus_utils.labelBox_utils
import json
import os


## Get the list of unique labels

In [4]:
def get_json_files(directory):
    json_files = [f for f in os.listdir(directory) if f.endswith('.json') and not f.startswith('._')]
    return json_files

unique_labels = set()

directories = [
    "/Volumes/MacFriendly/Annotation Stuff/MLFigs_Labeled_Oct_26_Chris",
    "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_03",
    "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_09_Chris",
    "/Volumes/MacFriendly/Annotation Stuff/MlFigsLabeled_Oct_25_Chris"
]

for directory in directories:

    json_files = get_json_files(directory)
    
    for json_file in json_files:

        with open(os.path.join(directory, json_file), 'r') as file:
            data = json.load(file)
            key_list = list(data.keys())

            for shape in data['shapes']:
                unique_labels.add(shape['label'])    
    
print(unique_labels)


{'fish', 'mooring', 'helicopter', 'airplane', 'boat', 'humpback', 'mooring noise', 'flow noise'}


## Get the unique list of annotation shape type

In [6]:
def get_json_files(directory):
    json_files = [f for f in os.listdir(directory) if f.endswith('.json') and not f.startswith('._')]
    return json_files

unique_shape_type = set()

directories = [
    "/Volumes/MacFriendly/Annotation Stuff/MLFigs_Labeled_Oct_26_Chris",
    "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_03",
    "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_09_Chris",
    "/Volumes/MacFriendly/Annotation Stuff/MlFigsLabeled_Oct_25_Chris"
]
polygon_counter = 0
for directory in directories:

    json_files = get_json_files(directory)
    #json_files_test = json_files[:1]

    #print(len(json_files_test))
    
    for json_file in json_files:

        with open(os.path.join(directory, json_file), 'r') as file:
            data = json.load(file)
            key_list = list(data.keys())
            #print(key_list)

            for shape in data['shapes']:
                #print(shape['shape_type'])
                # print(shape['points'])
                # print(shape['points'][0])
                # print(shape['points'][1])

                if shape['shape_type'] == 'polygon':
                    # print("in polygon")
                    # print(data)
                    polygon_counter = polygon_counter + 1

                unique_shape_type.add(shape['shape_type'])    
    
print(unique_shape_type)
print(polygon_counter)


{'rectangle', 'polygon'}
2


## Verify that all json files can be matched with a corresponding image file

In [12]:
def get_json_files(directory):
    json_files = [f for f in os.listdir(directory) if f.endswith('.json') and not f.startswith('._')]
    return json_files

directories = [
    "/Volumes/MacFriendly/Annotation Stuff/MLFigs_Labeled_Oct_26_Chris",
    "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_03",
    "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_09_Chris",
    "/Volumes/MacFriendly/Annotation Stuff/MlFigsLabeled_Oct_25_Chris"
]

not_found_images = []

for directory in directories:

    json_files = get_json_files(directory)
    #json_files_test = json_files[:1]

    #print(len(json_files_test))
    
    # for json_file in json_files_test:
    for json_file in json_files:

        with open(os.path.join(directory, json_file), 'r') as file:
            data = json.load(file)
            #key_list = list(data.keys())
            #print(key_list)

            image_path = data['imagePath']
            #print(f"image_path: {image_path}")

            # Construct the full path to the image file
            full_image_path = os.path.join(directory, image_path)

            # Check if the image file exists in the directory
            if os.path.exists(full_image_path):
                #print(f"Image file found: {full_image_path}")
                continue
            else:
                print(f"Image file not found: {full_image_path}")
                not_found_images.append(full_image_path)

            # for shape in data['shapes']:
            #     #print(shape['shape_type'])
            #     # print(shape['points'])
            #     # print(shape['points'][0])
            #     # print(shape['points'][1])

            #     if shape['shape_type'] == 'polygon':
            #         print("in polygon")
            #         print(data)

            #     else:
            #         print(data)

print(len(not_found_images))
# with open('not_found_images.txt', 'w') as f:
#     for path in not_found_images:
#         f.write(f"{path}\n")

# print("List of not found images saved to not_found_images.txt")

0


In [35]:
import os
import json

def get_json_files(directory):
    json_files = [f for f in os.listdir(directory) if f.endswith('.json') and not f.startswith('._')]
    return json_files

directories = [
    "/Volumes/MacFriendly/Annotation Stuff/MLFigs_Labeled_Oct_26_Chris",
    "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_03",
    "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_09_Chris",
    "/Volumes/MacFriendly/Annotation Stuff/MlFigsLabeled_Oct_25_Chris"
]

image_paths = []

for directory in directories:
    json_files = get_json_files(directory)
    for json_file in json_files:
        with open(os.path.join(directory, json_file), 'r') as file:
            data = json.load(file)
            image_path = data['imagePath']
            full_image_path = os.path.join(directory, image_path)
            image_paths.append(image_path)

# Finding duplicates
path_counts = {}
for path in image_paths:
    path_counts[path] = path_counts.get(path, 0) + 1

duplicates = [path for path, count in path_counts.items() if count > 1]

# Printing the results
if duplicates:
    print("Duplicate image paths found:")
    print(len(duplicates))
    for duplicate in duplicates:
        print(duplicate)
else:
    print("No duplicate image paths found.")


Duplicate image paths found:
305
20190217T023004-File-27.png
20190215T220004-File-25.png
20190213T040004-File-28.png
20190213T040004-File-17.png
20190212T023004-File-17.png
20190210T173004-File-25.png
20190210T160004-File-9.png
20190210T053004-File-26.png
20190210T040004-File-4.png
20190210T023004-File-16.png
20190209T173004-File-11.png
20190209T100004-File-10.png
20190208T010004-File-14.png
20190207T203004-File-26.png
20190204T010004-File-10.png
20190203T190004-File-25.png
20190201T143004-File-28.png
20190201T143004-File-14.png
20190201T130004-File-25.png
20190201T113004-File-2.png
20190201T083004-File-7.png
20190131T233004-File-2.png
20190131T220004-File-25.png
20190131T173004-File-27.png
20190131T100004-File-7.png
20190131T070004-File-7.png
20190131T053004-File-19.png
20190131T053004-File-18.png
20190131T053004-File-9.png
20190129T173004-File-24.png
20190129T173004-File-12.png
20190129T160004-File-7.png
20190129T143004-File-12.png
20190129T130004-File-22.png
20190129T113004-File-25.

## Add annotatations to original-annotations dataset in Labelbox

In [12]:
import uuid
import requests
import labelbox as lb
import labelbox.types as lb_types

In [14]:
file_path_test = "/Volumes/MacFriendly/Annotation Stuff/MLFigs_Labeled_Oct_26_Chris/20181204T100004-File-0.json"

with open(file_path_test, 'r') as file:
    data = json.load(file)
    image_name = data['imagePath']
    print(image_name)

api_key = azutils.get_secret("yolov8-keys", "labelbox-djsdavis")

# API_KEY = ""
client = lb.Client(api_key)

dataset = client.get_dataset("clrk34z5q005w07555h8j8b2j")

# alternatively, you can get a dataset by name
#dataset = client.get_datasets(where=lb.Dataset.name == "<dataset_name>").get_one()

data_rows = dataset.data_rows()

# inspect one data row
# next(data_rows)

# # inspect a number of data rows
# for data_row in data_rows:
#   print(data_row)
  
# for ease of use, you can convert the paginated collection to a list
data_row_list = list(data_rows)
print(len(data_row_list))



20181204T100004-File-0.png
718


In [15]:
print(data_row_list[0])

<DataRow {
    "created_at": "2024-01-19 03:49:50.358000+00:00",
    "external_id": "20181204T100004-File-1.png",
    "global_key": null,
    "media_attributes": {
        "assetType": "image",
        "contentLength": 477395,
        "height": 1200,
        "mimeType": "image/png",
        "subType": "png",
        "superType": "image",
        "width": 2400
    },
    "metadata": [],
    "metadata_fields": [],
    "row_data": "https://storage.labelbox.com/clr1ayhtq02uc07vg0hyqhczu%2F3300a71d-3794-294e-a3b2-724d868dce12-20181204T100004-File-1.png?Expires=1705722964515&KeyName=labelbox-assets-key-3&Signature=8INm-vL7GNUoUG9eTm7lcK9pCmg",
    "uid": "clrk3qyes090b0773kwulpmc6",
    "updated_at": "2024-01-19 03:49:51.314000+00:00"
}>


In [32]:
external_id_counts = {}

for row in data_row_list:
    # Assuming each DataRow object has an attribute 'external_id'
    external_id = row.external_id
    
    if external_id in external_id_counts:
        external_id_counts[external_id] += 1
    else:
        external_id_counts[external_id] = 1

# Find all external_ids that occurred more than once
duplicates = [key for key, count in external_id_counts.items() if count > 1]

# Print the results
if duplicates:
    print("Duplicate external_id values found:")
    for d in duplicates:
        print(d)
else:
    print("No duplicate external_id values found.")

Duplicate external_id values found:
20181204T100004-File-1.png
20181204T113004-File-16.png
20181204T113004-File-26.png
20181204T143004-File-10.png
20181205T040004-File-21.png
20181205T070004-File-20.png
20181205T070004-File-28.png
20181205T083004-File-0.png
20181205T190004-File-25.png
20181205T233004-File-11.png
20181206T083004-File-4.png
20181206T100004-File-3.png
20181207T070004-File-20.png
20181207T220004-File-16.png
20181207T220004-File-24.png
20181208T010004-File-0.png
20181208T220004-File-14.png
20181209T023004-File-3.png
20181209T023004-File-14.png
20181209T040004-File-7.png
20181209T040004-File-21.png
20181209T053004-File-23.png
20181209T070004-File-6.png
20181209T083004-File-21.png
20181209T143004-File-11.png
20181209T173004-File-12.png
20181217T130004-File-5.png
20181218T173004-File-12.png
20181218T190004-File-24.png
20181218T233004-File-9.png
20181221T113004-File-1.png
20181223T053004-File-16.png
20181224T053004-File-18.png
20181227T083004-File-26.png
20181227T100004-File-6.

In [17]:
data_row_list_abbr = data_row_list[:3]
print(len(data_row_list_abbr))

3


In [24]:
print(type(data_row_list_abbr))

<class 'list'>


In [28]:
global_key_data_row_inputs = []

for row in data_row_list:
    # print(type(row))
    # print(row)
    
#     global_input_obj = {
#         "data_row_id" : row.uid,
#         "global_key" : row.external_id
#     }

#     global_key_data_row_inputs.append(global_input_obj)

# print(len(global_key_data_row_inputs))
#print(global_key_data_row_inputs)
    

718


In [31]:
print(global_key_data_row_inputs)

[{'data_row_id': 'clrk3qyes090b0773kwulpmc6', 'global_key': '20181204T100004-File-1.png'}, {'data_row_id': 'clrk3qyes090c0773n1f0ezqe', 'global_key': '20181204T113004-File-16.png'}, {'data_row_id': 'clrk3qyes090d0773pcolw43g', 'global_key': '20181204T113004-File-26.png'}, {'data_row_id': 'clrk3qyes090e0773f1hmhh0b', 'global_key': '20181204T143004-File-10.png'}, {'data_row_id': 'clrk3qyes090f0773w85w6ma4', 'global_key': '20181205T040004-File-21.png'}, {'data_row_id': 'clrk3qyes090g0773fcff59o7', 'global_key': '20181205T070004-File-20.png'}, {'data_row_id': 'clrk3qyes090h07732j0lqh0v', 'global_key': '20181205T070004-File-28.png'}, {'data_row_id': 'clrk3qyes090i0773nxoadim0', 'global_key': '20181205T083004-File-0.png'}, {'data_row_id': 'clrk3qyes090j07730w1m5rat', 'global_key': '20181205T190004-File-25.png'}, {'data_row_id': 'clrk3qyes090k0773cgwh17o3', 'global_key': '20181205T233004-File-11.png'}, {'data_row_id': 'clrk3qyes090l07733dzffc82', 'global_key': '20181206T083004-File-4.png'}, {

In [30]:
# global_key_data_row_inputs = [
#   {"data_row_id": "clrk3qyes090b0773kwulpmc6", "global_key": "20181204T100004-File-1.png"},
# ]

client.assign_global_keys_to_data_rows(global_key_data_row_inputs)

There are errors present. Please look at 'errors' in the returned dict for more details


{'status': 'PARTIAL SUCCESS',
 'results': [{'data_row_id': 'clrk3iq75105u0737rekkrrio',
   'global_key': '20190220T143004-File-2.png',
   'sanitized': False},
  {'data_row_id': 'clrk3iq75105v0737trybhqqm',
   'global_key': '20190221T083004-File-3.png',
   'sanitized': False},
  {'data_row_id': 'clrk3iq75105w0737nstjnz2q',
   'global_key': '20190221T100004-File-3.png',
   'sanitized': False},
  {'data_row_id': 'clrk3iq75105x0737n2q0mxfk',
   'global_key': '20190221T100004-File-10.png',
   'sanitized': False},
  {'data_row_id': 'clrk3iq75105y0737gm0hbof8',
   'global_key': '20190221T100004-File-13.png',
   'sanitized': False},
  {'data_row_id': 'clrk3iq75105z0737n3rdduc7',
   'global_key': '20190221T100004-File-15.png',
   'sanitized': False},
  {'data_row_id': 'clrk3iq7510600737u4vlf3cr',
   'global_key': '20190221T100004-File-17.png',
   'sanitized': False},
  {'data_row_id': 'clrk3iq7510610737iscv3cqj',
   'global_key': '20190221T100004-File-20.png',
   'sanitized': False},
  {'data_r

In [ ]:
label_ndjson = []
annotations = [
    radio_annotation_ndjson,
    nested_radio_annotation_ndjson,
    nested_checklist_annotation_ndjson,
    checklist_annotation_ndjson,
    text_annotation_ndjson,
    bbox_annotation_ndjson,
    bbox_with_radio_subclass_ndjson,
    polygon_annotation_ndjson,
    mask_annotation_ndjson,
    mask_with_text_subclass_ndjson,
    point_annotation_ndjson,
    polyline_annotation_ndjson,
    bbox_source_ndjson,
    bbox_target_ndjson,
    relationship_ndjson,  ## Only supported for MAL imports
]
for annotation in annotations:
    annotation.update({
        "dataRow": {
            "globalKey": global_key
        }
    })
    label_ndjson.append(annotation)

# upload MAL labels for this data row in project
upload_job = lb.MALPredictionImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="mal_job" + str(uuid.uuid4()),
    predictions=label
)
upload_job.wait_until_done()

print(f"Errors: {upload_job.errors}")
print(f"Status of uploads: {upload_job.statuses}")

## Upload image files to Labelbox

In [7]:
import uuid
import requests
import labelbox as lb
import labelbox.types as lb_types

In [5]:
api_key = azutils.get_secret("yolov8-keys", "labelbox-djsdavis")

# API_KEY = ""
client = lb.Client(api_key)

In [5]:
    # hydannotations-connection-string
blob_connection_string = azutils.get_secret("yolov8-keys", "hydannotations-connection-string")
#print(blob_connection_string)

blob_img_list = azutils.get_blob_file_list(blob_connection_string, "original-annotations")
print(blob_img_list)

['20181204T100004-File-0.png', '20181204T100004-File-1.png', '20181204T100004-File-10.png', '20181204T100004-File-17.png', '20181204T100004-File-21.png', '20181204T100004-File-27.png', '20181204T100004-File-8.png', '20181204T113004-File-10.png', '20181204T113004-File-11(2).png', '20181204T113004-File-11.png', '20181204T113004-File-15.png', '20181204T113004-File-16.png', '20181204T113004-File-26.png', '20181204T113004-File-9.png', '20181204T130004-File-3.png', '20181204T143004-File-10.png', '20181204T143004-File-11.png', '20181204T143004-File-26.png', '20181204T143004-File-27.png', '20181204T160004-File-23.png', '20181204T160004-File-24.png', '20181204T173004-File-11.png', '20181204T173004-File-24.png', '20181204T173004-File-5.png', '20181204T190004-File-13.png', '20181204T190004-File-21.png', '20181204T203004-File-0.png', '20181204T203004-File-20.png', '20181204T203004-File-29.png', '20181205T010004-File-21.png', '20181205T023004-File-22.png', '20181205T040004-File-21.png', '20181205T0

In [7]:
def get_json_files(directory):
    json_files = [f for f in os.listdir(directory) if f.endswith('.json') and not f.startswith('._')]
    return json_files

directories = [
    "/Volumes/MacFriendly/Annotation Stuff/MLFigs_Labeled_Oct_26_Chris",
    # "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_03",
    # "/Volumes/MacFriendly/Annotation Stuff/MLFigsLabeled_Nov_09_Chris",
    # "/Volumes/MacFriendly/Annotation Stuff/MlFigsLabeled_Oct_25_Chris"
]

not_found_images = []
blob_directory = "https://hydannotations.blob.core.windows.net/original-annotations/"

for directory in directories:

    json_files = get_json_files(directory)
    #json_files_test = json_files[:1]

    #print(len(json_files_test))
    
    # for json_file in json_files_test:
    for json_file in json_files:

        with open(os.path.join(directory, json_file), 'r') as file:
            data = json.load(file)
            #key_list = list(data.keys())
            #print(key_list)
            image_name = data['imagePath']
            if image_name in blob_img_list:
                blob_url = blob_directory + image_name
                #print(blob_url)
                upload_img_labelbox(blob_url)
                continue
            else:
                print(f"Image file not found: {full_image_path}")
                not_found_images.append(full_image_path)
            
            #print(f"image_path: {image_path}")

            # Construct the full path to the image file


            # for shape in data['shapes']:
            #     #print(shape['shape_type'])
            #     # print(shape['points'])
            #     # print(shape['points'][0])
            #     # print(shape['points'][1])

            #     if shape['shape_type'] == 'polygon':
            #         print("in polygon")
            #         print(data)

            #     else:
            #         print(data)

print(len(not_found_images))
# with open('not_found_images.txt', 'w') as f:
#     for path in not_found_images:
#         f.write(f"{path}\n")

# print("List of not found images saved to not_found_images.txt")

https://hydannotations.blob.core.windows.net/original-annotations/20190223T083004-File-18.png
https://hydannotations.blob.core.windows.net/original-annotations/20190223T083004-File-12.png
https://hydannotations.blob.core.windows.net/original-annotations/20190222T190004-File-28.png
https://hydannotations.blob.core.windows.net/original-annotations/20190222T113004-File-28.png
https://hydannotations.blob.core.windows.net/original-annotations/20190222T100004-File-19.png
https://hydannotations.blob.core.windows.net/original-annotations/20190222T083004-File-15.png
https://hydannotations.blob.core.windows.net/original-annotations/20190222T083004-File-2.png
https://hydannotations.blob.core.windows.net/original-annotations/20190222T070004-File-24.png
https://hydannotations.blob.core.windows.net/original-annotations/20190221T143004-File-1.png
https://hydannotations.blob.core.windows.net/original-annotations/20190221T130004-File-22.png
https://hydannotations.blob.core.windows.net/original-annotati

## IGNORE BELOW HERE FOR NOW

In [ ]:
def create_labelbox_annotation(shape, data_row_id, schema_id_mapping):
    # Assuming the 'points' are top-left and bottom-right for a rectangle
    start = shape['points'][0]
    stop = shape['points'][1]
    
    #   "shapes": [
    # {
    #   "label": "mooring",
    #   "points": [
    #     [
    #       322.18518518518516, # top left
    #       592.1111111111111
    #     ],
    #     [
    #       388.85185185185185, # bottom right
    #       737.2962962962962
    #     ]
    #   ],

    1
# [[536.7831325301204, 1030.1445783132529], [965.6987951807228, 562.6746987951807]]
# [[1007.867469879518, 526.5301204819276], [1072.9277108433735, 643.3975903614457]]
# [[1082.5662650602408, 527.7349397590361], [1157.2650602409637, 648.2168674698794]]
# [[1369.313253012048, 948.2168674698794], [1431.9638554216867, 624.1204819277108]]
# [[1497.024096385542, 968.6987951807228], [1553.6506024096384, 765.0843373493975]]
# [[1606.6626506024095, 978.3373493975903], [1569.313253012048, 606.0481927710842]]
# [[1636.7831325301204, 975.9277108433735], [1690.9999999999998, 860.2650602409637]]
# [[1711.4819277108431, 875.9277108433735], [1756.0602409638552, 1000.024096385542]]
# [[1776.5421686746986, 951.8313253012047], [1818.7108433734938, 868.6987951807228]]
# [[1856.0602409638552, 855.4457831325301], [1886.1807228915661, 959.0602409638552]]
# [[1912.6867469879517, 951.8313253012047], [1959.6746987951806, 839.7831325301204]]
# [[2086.180722891566, 542.1927710843373], [2139.192771084337, 602.433734939759]]

    # bbox_with_radio_subclass_annotation = lb_types.ObjectAnnotation(
    # name="bbox_with_radio_subclass",
    # value=lb_types.Rectangle(
    #     start=lb_types.Point(x=541, y=933),  # x = left, y = top 
    #     end=lb_types.Point(x=871, y=1124),  # x= left + width , y = top + height
    # ),
    # classifications=[
    #     lb_types.ClassificationAnnotation(
    #         name="sub_radio_question",
    #         value=lb_types.Radio(answer=lb_types.ClassificationAnswer(
    #             name="first_sub_radio_answer")))
    # ])
    
    return {
        'uuid': str(uuid.uuid4()),
        'schemaId': schema_id_mapping.get(shape['label'], 'YOUR_DEFAULT_SCHEMA_ID'),
        'dataRow': {'id': data_row_id},
        'bbox': {
            'top': top_left[1],
            'left': top_left[0],
            'height': height,
            'width': width
        }
    }

In [ ]:
# Schema ID mapping (you need to replace these with actual schema IDs from Labelbox)
schema_id_mapping = {
    'mooring': 'YOUR_SCHEMA_ID_FOR_MOORING'
}

# Data row ID (replace with your actual dataRowId from Labelbox)
data_row_id = 'YOUR_DATAROW_ID'

# Load the original JSON file
file_path = 'path_to_your_json_file.json'
with open(file_path, 'r') as file:
    original_json_data = json.load(file)

# Convert the data to Labelbox format
converted_data = labelBox_utils.convert_to_labelbox_format(original_json_data, schema_id_mapping, data_row_id)

# Save the converted data to a new JSON file
output_file_path = 'path_to_your_output_json_file.json'
with open(output_file_path, 'w') as file:
    json.dump(converted_data, file, indent=4)

print(f"Converted data saved to {output_file_path}")